## model training data

In [31]:
import nltk 
import string 
import re 
import pandas as pd
import boto3
import pickle

In [5]:
#import data

In [6]:
df_inroads = pd.read_csv('https://inroads-test-bucket1.s3.amazonaws.com/inroads_only.csv', dtype={'text': str})
#df_raw = pd.read_csv('https://inroads-test-bucket1.s3.amazonaws.com/raw_data.csv', dtype={'text': str})

In [7]:
text_to_clean = df_inroads

In [8]:
# preprocessing functions

In [32]:
def text_lowercase(text): 
    return text.lower() 

In [33]:
def remove_numbers(text): 
    result = re.sub(r'\d+', '', text) 
    return result 

In [34]:
def remove_punctuation(text):
    translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    temp = text.translate(translator)
    return re.sub('\s+',' ', temp)

In [35]:
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize 
lemmatizer = WordNetLemmatizer() 
nltk.download("punkt")
nltk.download('wordnet')
# lemmatize string 

def lemmatize_word(text): 
    word_tokens = word_tokenize(text) 
    # provide context i.e. part-of-speech 
    lemmas = [lemmatizer.lemmatize(word, pos ='v') for word in word_tokens] 
    return lemmas 

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ec2-user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [36]:
def remove_short_words(text):
    clean_words = []
    for word in text:
        new = re.sub(r'\b\w{,3}\b', '', word)
        clean_words.append(new) 
    return clean_words

In [14]:
result = list()

for i in range(0,len(text_to_clean)):
    temp = str(text_to_clean.text.iloc[i])     
    temp = text_lowercase(temp) #make all lower
    temp_2 = remove_numbers(temp) # remove all numbers
    temp_3 = str.strip(temp_2) # remove white spaces 1
    temp_4 = temp_3.replace('\n',' ').replace("\t", " ") # remove white spaces 2
    temp_5 = re.sub(" +", " ", temp_4) # remove white spaces 3
    temp_6 = remove_punctuation(temp_5) # remove punctuation 
    temp_7 = lemmatize_word(temp_6) # lemmatize text
    temp_8 = [word for word in temp_7 if word.isalnum()] # remove auxiliarly punctuation
    temp_9 = remove_short_words(temp_8) # remove short words
    result.append(list(filter(None, temp_9))) # remove empty strings and save

In [15]:
text_to_clean['clean_text'] = result

In [16]:
## review

In [17]:
text_to_clean.head

<bound method NDFrame.head of       Unnamed: 0                                           resource  \
0              1  649607\nresearch-article2016\n                ...   
1              2  649607\nresearch-article2016\n                ...   
2              3  649607\nresearch-article2016\n                ...   
3              4  649607\nresearch-article2016\n                ...   
4              5  649607\nresearch-article2016\n                ...   
...          ...                                                ...   
1221        1222  the-inroads-team-is-growing-introducing-our-ne...   
1222        1223  there-is-no-qualification-to-be-here-just-pass...   
1223        1224                                           wfac.txt   
1224        1225  what-is-inspiring-our-steering-committee-in-20...   
1225        1226  young-people-fight-societal-norms-in-sudan-whe...   

                                                   text  \
0     649607\nresearch-article2016\n                ...   

In [18]:
text_to_clean.clean_text[:100]

0     [research, article, pspxxx, personality, socia...
1     [huang, describe, current, program, research, ...
2     [personality, social, psychology, bulletin, mo...
3     [huang, abortion, attitudes, over, year, perio...
4     [table, descriptive, statistics, bivariate, co...
                            ...                        
95    [towards, asps, show, figure, informants, expr...
96    [behavior, community, towards, people, associa...
97    [abortion, brother, stop, talk, with, after, l...
98    [person, support, abortion, spaw, fgds, partic...
99    [women, exclude, religious, practice, abortive...
Name: clean_text, Length: 100, dtype: object

In [19]:
## save cleaned tokens for further use

In [20]:
final_inroads_clean = text_to_clean[['resource', 'clean_text']].copy()

In [21]:
final_inroads_clean.head

<bound method NDFrame.head of                                                resource  \
0     649607\nresearch-article2016\n                ...   
1     649607\nresearch-article2016\n                ...   
2     649607\nresearch-article2016\n                ...   
3     649607\nresearch-article2016\n                ...   
4     649607\nresearch-article2016\n                ...   
...                                                 ...   
1221  the-inroads-team-is-growing-introducing-our-ne...   
1222  there-is-no-qualification-to-be-here-just-pass...   
1223                                           wfac.txt   
1224  what-is-inspiring-our-steering-committee-in-20...   
1225  young-people-fight-societal-norms-in-sudan-whe...   

                                             clean_text  
0     [research, article, pspxxx, personality, socia...  
1     [huang, describe, current, program, research, ...  
2     [personality, social, psychology, bulletin, mo...  
3     [huang, abortion, attit

In [22]:
final_inroads_clean.to_pickle('final_inroads_clean.pkl')
final_inroads_clean = pd.read_pickle('final_inroads_clean.pkl')

In [19]:
final_inroads_clean = pd.read_pickle("final_inroads_clean.pkl")
clean_inroads_tokens = []

for i in range(0,len(final_inroads_clean)):
    text = final_inroads_clean.clean_text[i]
    #print(text)
    clean_inroads_tokens.extend(text)

In [20]:
clean_inroads_tokens = pd.DataFrame(clean_inroads_tokens,columns=['tokens'])
clean_inroads_tokens.head

<bound method NDFrame.head of              tokens
0          research
1           article
2            pspxxx
3       personality
4            social
...             ...
321033       slowly
321034        there
321035        space
321036        start
321037   discussion

[321038 rows x 1 columns]>

In [26]:
clean_inroads_tokens.to_pickle('clean_inroads_tokens.pkl')

In [61]:
clean_inroads_tokens = pd.read_pickle('clean_inroads_tokens.pkl')

In [212]:
## upload trainign data

In [213]:
import boto3
bucket = 'sagemaker-word2vec-scikitlearn'
region = 'us-east-2'
s3_session = boto3.Session().resource('s3')
s3_session.create_bucket(Bucket=bucket, 
                         CreateBucketConfiguration=
                         {'LocationConstraint': region})
s3_session.Bucket(bucket).Object('train/clean_full_tokens.pkl').upload_file('clean_full_tokens.pkl')

BucketAlreadyOwnedByYou: An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.

In [214]:
## create dictionary

In [215]:
word_dict = clean_inroads_tokens.to_dict() 

In [216]:
word_dict

{'tokens': {0: 'research',
  1: 'article',
  2: 'pspxxx',
  3: 'personality',
  4: 'social',
  5: 'psychology',
  6: 'bulletinhuang',
  7: 'article',
  8: 'personality',
  9: 'social',
  10: 'benevolent',
  11: 'sexism',
  12: 'attitudes',
  13: 'toward',
  14: 'psychology',
  15: 'bulletin',
  16: 'society',
  17: 'personality',
  18: 'motherhood',
  19: 'reproductive',
  20: 'right',
  21: 'social',
  22: 'psychology',
  23: 'reprint',
  24: 'permissions',
  25: 'multi',
  26: 'study',
  27: 'longitudinal',
  28: 'examination',
  29: 'sagepub',
  30: 'journalspermissions',
  31: 'abortion',
  32: 'attitudes',
  33: 'pspb',
  34: 'sagepub',
  35: 'yanshu',
  36: 'huang',
  37: 'paul',
  38: 'davies',
  39: 'chris',
  40: 'sibley',
  41: 'danny',
  42: 'osborne',
  43: 'abstract',
  44: 'although',
  45: 'benevolent',
  46: 'sexism',
  47: 'ideology',
  48: 'that',
  49: 'highly',
  50: 'revere',
  51: 'women',
  52: 'conform',
  53: 'traditional',
  54: 'gender',
  55: 'cloak',
  56: 

In [64]:
data_dir = 'dict_data/word2vec' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    os.makedirs(data_dir)

In [71]:
with open(os.path.join(data_dir, 'word_dict.pkl'), "wb") as f:
    pickle.dump(word_dict, f)

## old user data for training

In [4]:
old_data = pd.read_csv('https://inroads-test-bucket1.s3.amazonaws.com/Taxonomy-Grid+view+(1).csv', dtype={'text': str})

In [5]:
old_data.head

<bound method NDFrame.head of                RecordID       Term      Domain      Subdomain
0     rec8EgTumvHTEJdLa   abstract  Conceptual     Conceptual
1     recMJFlgfo00roAuw    absence  Conceptual     Conceptual
2     recAuJIeX88OP6ZIb  abundance  Conceptual     Conceptual
3     recz0CPCI09cP5iUX    reality  Conceptual     Conceptual
4     rec8Bm0aJwpSKdOGQ  beautiful  Conceptual     Conceptual
...                 ...        ...         ...            ...
1016  recMKWuPq8dMo70HD     summer    Temporal  Units of time
1017  recjrhx9mX0O1xCu0     autumn    Temporal  Units of time
1018  recraEJIJ7Fe5acU5       1971    Temporal  Units of time
1019  recvyH9ENCss8n1xG       1972    Temporal  Units of time
1020  recKH4SX6UXD9wvKv        NaN         NaN            NaN

[1021 rows x 4 columns]>

In [6]:
old_data = old_data[['Term','Domain']].copy().sort_values(['Term','Domain'])

In [30]:
old_data.to_csv('inroads_old_data.csv')
old_data.to_pickle('inroads_old.pkl')

In [15]:
## clean

In [13]:
result = list()

for i in range(0,len(old_data)):
    temp = str(old_data.Term.iloc[i])     
    temp = text_lowercase(temp) #make all lower
    temp_2 = remove_numbers(temp) # remove all numbers
    temp_3 = str.strip(temp_2) # remove white spaces 1
    temp_4 = temp_3.replace('\n',' ').replace("\t", " ") # remove white spaces 2
    temp_5 = re.sub(" +", " ", temp_4) # remove white spaces 3
    temp_6 = remove_punctuation(temp_5) # remove punctuation 
    temp_7 = lemmatize_word(temp_6) # lemmatize text
    temp_8 = [word for word in temp_7 if word.isalnum()] # remove auxiliarly punctuation
    temp_9 = remove_short_words(temp_8) # remove short words
    result.append(list(filter(None, temp_9))) # remove empty strings and save

In [14]:
old_data['clean_text'] = result

In [15]:
old_data.clean_text.head()

724              []
1018             []
1019             []
928     [september]
682              []
Name: clean_text, dtype: object

In [16]:
old_tokens = []

for i in range(0,len(old_data)):
    text = old_data.clean_text[i]
    #print(text)
    old_tokens.extend(text)

In [17]:
old_tokens = pd.DataFrame(old_tokens,columns=['tokens'])
old_tokens.head

<bound method NDFrame.head of          tokens
0      abstract
1       absence
2     abundance
3       reality
4     beautiful
...         ...
1307     winter
1308     spring
1309     season
1310     summer
1311     autumn

[1312 rows x 1 columns]>

In [36]:
old_tokens.to_pickle('old_tokens.pkl')

## train word2vec


In [37]:
!pip install gensim
from gensim.models.word2vec import Word2Vec
from sklearn.cluster import KMeans;
#from sklearn.neighbors import KDTree
import numpy as np

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [9]:
#token_list = list(final_inroads_clean.clean_text)
token_list = pd.read_pickle("final_inroads_clean.pkl").clean_text

you can use the same model I use, which is saved, I provide code to download it from s3.  Or, you can create your own by uncommenting the training code.

In [119]:
#num_features = 300
#min_word_count = 50
#num_workers = 2
#window_size = 6
#subsampling = 1e-3

In [122]:
#model = Word2Vec(
#    token_list, 
#    workers = num_workers, 
#    size = num_features, 
#    min_count=min_word_count, 
#    window=window_size, 
#    sample=subsampling)

we need both pieces to make the model work, I'm struggling to find a way for me to upload and download the gensim model from S3.  TO BE DETERMINED

In [429]:
import boto
from boto.s3.connection import OrdinaryCallingFormat


S3_BUCKET = 'inroads-test-bucket1'
S3_KEY = 'w2v_model.bin'

def download_gensim_model():
    s3_conn = boto3.resource('s3', aws_access_key_id=os.environ.get('AWS_ACCESS_KEY_ID'),
                    aws_secret_access_key=os.environ.get('AWS_SECRET_ACCESS_KEY'))
    bucket = s3_conn.Bucket(S3_BUCKET)
    #key = s3_conn.get_key(S3_KEY)
    model = gensim.models.Word2Vec.load_word2vec_format(key, binary=True)
    return model

model = download_gensim_model()

ModuleNotFoundError: No module named 'boto3.s3.connection'

In [427]:
import boto3
s3 = boto3.resource('s3')
bucket = s3.Bucket('inroads-test-bucket1')
for obj in bucket.objects.all():
    print(obj.key, obj.last_modified)

Taxonomy-Grid view (1).csv 2020-08-14 16:05:03+00:00
clean_full_text.txt 2020-08-14 00:54:00+00:00
clean_text.txt 2020-07-26 19:23:38+00:00
clean_tokens.txt 2020-07-26 19:24:58+00:00
cleaned_data.pkl 2020-07-16 20:34:30+00:00
clusters/ 2020-07-27 22:01:22+00:00
clusters/top_words_10.csv 2020-07-27 23:23:35+00:00
clusters/top_words_11.csv 2020-07-27 23:23:35+00:00
clusters/top_words_12.csv 2020-07-27 23:23:35+00:00
clusters/top_words_13.csv 2020-07-27 23:23:35+00:00
clusters/top_words_4.csv 2020-07-27 23:23:35+00:00
clusters/top_words_5.csv 2020-07-27 23:23:35+00:00
clusters/top_words_6.csv 2020-07-27 23:23:35+00:00
clusters/top_words_7.csv 2020-07-29 16:29:33+00:00
clusters/top_words_8.csv 2020-07-29 16:29:33+00:00
clusters/top_words_9.csv 2020-07-27 23:23:35+00:00
inroads_blog.csv 2020-06-17 18:36:33+00:00
inroads_only.csv 2020-06-17 18:41:20+00:00
inroads_word2vec_model 2020-08-17 14:23:03+00:00
inroads_word2vec_model.wv.vectors.npy 2020-08-17 14:23:03+00:00
raw_data.csv 2020-06-04 2

In [436]:
#s3.Bucket('inroads-test-bucket1').download_file('w2v_model.bin', 'model_2.bin')
#s3.Bucket('inroads-test-bucket1').download_file('inroads_word2vec_model', 'model_2')

TypeError: head_object() argument after ** must be a mapping, not _io.BufferedRandom

In [38]:
model_name = "inroads_word2vec_model"
model = Word2Vec.load(model_name)

ValueError: cannot reshape array of size 10610656 into shape (35436,300)

In [11]:
model.wv.most_similar("abortion")

[('abortions', 0.5601829290390015),
 ('postabortion', 0.4012739658355713),
 ('contraception', 0.3701522946357727),
 ('abor', 0.36626535654067993),
 ('medically', 0.34925901889801025),
 ('antenatal', 0.3442828059196472),
 ('termination', 0.3441293239593506),
 ('reproductive', 0.3407558798789978),
 ('misoprostol', 0.338686466217041),
 ('contraceptive', 0.33505553007125854)]

In [21]:
def get_vector(text):
    vector_list = []
    
    for i in range(len(text)):
        word = text[i]     
        try: 
            vector = model.wv[word]
        except KeyError:
            pass
        else:
            if vector is None:
                vector = np.zeros(300,dtype=int) 
                vector_list.append([word, vector])
            else: 
                vector_list.append([word, vector])
        #vector_embed.append(pd.Series([word, vector], index=['token',range(0,300)]), ignore_index=True)
                    
    return vector_list 

In [24]:
old_tokens = pd.read_pickle("old_tokens.pkl")
token_list_old = list(old_tokens.tokens)
token_list_old = list(dict.fromkeys(token_list_old))
len(token_list_old)

353

In [25]:
old_vectors = get_vector(token_list_old)

old_vocab = list()

for i in range(0,len(old_vectors)):
    test = old_vectors[i]
    old_vocab.append(test[0])

old_vocab_df = pd.DataFrame(old_vocab, columns=["token"])

old_vectors_2 = list()

for i in range(0,len(old_vectors)):
    test = old_vectors[i]
    old_vectors_2.append(test[1])
    
old_array = np.array(old_vectors_2)
old_vectors_df = pd.DataFrame(old_vectors, columns=["token", "vector"])

old_vectors_df_2 = pd.DataFrame(old_vectors_df.vector.tolist())
old_vectors_df_2.head

result = pd.concat([old_vectors_df, old_vectors_df_2], axis=1)
result.head

NameError: name 'model' is not defined

In [43]:
old_vectors_df_2[0:100]

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.009340,-0.011540,-0.005011,0.170543,-0.070042,-0.022673,0.035479,-0.033519,0.038251,0.045745,...,0.027858,0.024611,-0.013685,0.042488,0.054621,-0.036383,0.018394,-0.045638,0.025203,-0.052491
1,0.073121,-0.047603,0.091147,0.162190,-0.052149,-0.108441,-0.028311,-0.041613,-0.026646,-0.015232,...,0.018856,0.007211,-0.080637,0.157568,-0.035902,-0.001056,-0.014947,-0.114950,-0.121481,0.031619
2,0.013254,0.050222,-0.089445,0.083483,-0.044747,-0.065899,0.019941,-0.052136,0.037055,-0.078439,...,0.043838,0.002926,-0.050873,-0.012946,-0.007152,0.148603,0.109223,-0.019303,0.029999,0.042774
3,0.017232,-0.138262,-0.069550,0.161501,-0.024385,-0.121781,-0.035268,0.007963,0.047652,-0.019447,...,0.027821,0.020316,-0.028523,0.028286,-0.049850,-0.049433,0.133848,0.008867,-0.076711,0.042498
4,-0.056970,0.020302,-0.024677,-0.048865,0.064803,-0.033035,-0.029016,-0.020143,0.038997,0.015680,...,-0.071911,-0.002086,0.023876,0.077053,0.024279,-0.043584,-0.053606,0.007344,0.018627,-0.046889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.013136,-0.014611,-0.012390,0.006732,-0.045401,0.076660,-0.069420,-0.037662,0.050937,-0.020486,...,0.029552,0.051054,0.041030,-0.028855,-0.004319,0.048255,-0.001835,0.036622,0.038000,-0.002147
96,-0.086506,-0.052345,-0.011814,-0.034647,-0.079073,-0.058749,-0.012539,0.001059,-0.086351,-0.025194,...,0.012264,0.015519,-0.025577,0.011025,0.039866,0.007830,0.045241,-0.003326,0.046087,-0.122829
97,-0.123545,-0.013735,-0.113005,-0.184742,-0.081858,-0.060789,0.017910,0.059038,-0.041535,-0.106276,...,0.107440,0.065636,0.106605,-0.057781,0.029954,0.081753,0.071285,-0.078630,0.021454,-0.010050
98,-0.114382,0.004027,-0.012011,-0.072564,0.026916,-0.040016,0.049948,-0.069554,0.014562,-0.100948,...,-0.014017,-0.057527,0.088652,-0.042609,-0.056985,-0.040120,-0.005427,-0.015367,-0.112244,0.004891


In [39]:
bucket = 'inroads-test-bucket1' # already created on S3
csv_buffer = StringIO()
result.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'data/Vectorized/old_vectorized.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '8A9BFFEF6CF3F426',
  'HostId': 'vNuviOXT1MKQmw+As1f8Z8H9dSiZaSdShIMR/l0qdqWau7pu36f1Drtj7OVh2SbZ5Wdo7Rm+ZGs=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'vNuviOXT1MKQmw+As1f8Z8H9dSiZaSdShIMR/l0qdqWau7pu36f1Drtj7OVh2SbZ5Wdo7Rm+ZGs=',
   'x-amz-request-id': '8A9BFFEF6CF3F426',
   'date': 'Wed, 19 Aug 2020 20:18:49 GMT',
   'etag': '"4126fa3c0f3f0829b135695480016e51"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"4126fa3c0f3f0829b135695480016e51"'}

In [21]:
clean_inroads_tokens = pd.read_pickle("clean_inroads_tokens.pkl")
token_list_inroads = list(clean_inroads_tokens.tokens)
token_list_inroads = list(dict.fromkeys(token_list_inroads))
len(token_list_inroads)

13196

In [22]:
inroads_vectors = get_vector(token_list_inroads)

In [36]:

inroads_vocab = list()

for i in range(0,len(inroads_vectors)):
    test = inroads_vectors[i]
    inroads_vocab.append(test[0])

inroads_vocab_df = pd.DataFrame(inroads_vocab, columns=["tokens"])

inroads_vectors_2 = list()

for i in range(0,len(inroads_vectors)):
    test = inroads_vectors[i]
    inroads_vectors_2.append(test[1])
    
inroads_array = np.array(inroads_vectors_2)
inroads_vectors_df = pd.DataFrame(inroads_vectors, columns=["tokens", "vector"])

inroads_vectors_df_2 = pd.DataFrame(inroads_vectors_df.vector.tolist())
inroads_vectors_df_2.head

inroads_result = pd.concat([inroads_vectors_df, inroads_vectors_df_2], axis=1)
inroads_result.head

<bound method NDFrame.head of              tokens                                             vector  \
0          research  [0.18854871, -0.0510548, -0.051034868, 0.01212...   
1           article  [-0.08915062, -0.032444216, 0.0660668, -0.0830...   
2       personality  [0.080988534, 0.07145718, -0.036697134, 0.0516...   
3            social  [0.04818078, -0.024814632, -0.042576127, 0.044...   
4        psychology  [-0.0020839642, 0.08448851, 0.014705653, 0.073...   
...             ...                                                ...   
10056         saudi  [-0.014218387, -0.024083603, 0.08134469, -0.06...   
10057        arabia  [-0.07335844, -0.063884735, 0.05745144, -0.059...   
10058      adultery  [-0.08255755, 0.04301242, -0.11898199, -0.1686...   
10059      indecent  [0.010403189, -0.0015966488, 0.022658184, 0.14...   
10060  islamisation  [0.013339531, 0.02428078, -0.089154445, 0.0554...   

              0         1         2         3         4         5         6  \
0 

In [37]:
from io import StringIO # python3; python2: BytesIO 
import boto3

bucket = 'inroads-test-bucket1' # already created on S3
csv_buffer = StringIO()
inroads_result.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, 'data/Vectorized/inroads_vectorized.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '5CD4F7D5ECF7076B',
  'HostId': 'B+1tY77Hez35juGkeVk1rqhVBF15VfEszClKiqnUQ/p0DSpvHpuwGCI3H5jFLaOhhIqkXZTdirM=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'B+1tY77Hez35juGkeVk1rqhVBF15VfEszClKiqnUQ/p0DSpvHpuwGCI3H5jFLaOhhIqkXZTdirM=',
   'x-amz-request-id': '5CD4F7D5ECF7076B',
   'date': 'Wed, 19 Aug 2020 20:16:14 GMT',
   'etag': '"f89778b8ce47e286e8ef7291c068304c"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"f89778b8ce47e286e8ef7291c068304c"'}

## Cluster

In [74]:
def clustering_on_wordvecs(word_vectors, num_clusters):
    # Initalize a k-means object and use it to extract centroids
    kmeans_clustering = KMeans(n_clusters = num_clusters, init='k-means++');
    idx = kmeans_clustering.fit_predict(word_vectors);
    
    return kmeans_clustering, kmeans_clustering.cluster_centers_, idx;

In [67]:
def get_top_words(index2word, k, centers, wordvecs):
    tree = KDTree(wordvecs);
#Closest points for each Cluster center is used to query the closest 20 points to it.
    closest_points = [tree.query(np.reshape(x, (1, -1)), k=k) for x in centers];
    closest_words_idxs = [x[1] for x in closest_points];
#Word Index is queried for each position in the above array, and added to a Dictionary.
    closest_words = {};
    for i in range(0, len(closest_words_idxs)):
        closest_words['Cluster #' + str(i)] = [index2word[j] for j in closest_words_idxs[i][0]]
#A DataFrame is generated from the dictionary.
    df = pd.DataFrame(closest_words);
    df.index = df.index+1
    return df, tree;

In [75]:
kmodel,centers,clusters = clustering_on_wordvecs(old_vectors_2, 9);
centroid_map = dict(zip(old_vocab, clusters));

In [84]:
predicted_cluster = {}

for i in range(0, len(inroads_vectors_2)):
    word = inroads_vocab[i]
    temp = kmodel.predict(inroads_vectors_2[i].reshape(1,-1))[0]
    predicted_cluster.update({word:temp})

In [93]:
predicted_cluster

{'research': 0,
 'article': 0,
 'personality': 0,
 'social': 5,
 'psychology': 0,
 'benevolent': 5,
 'sexism': 6,
 'attitudes': 0,
 'toward': 5,
 'bulletin': 4,
 'society': 5,
 'motherhood': 0,
 'reproductive': 6,
 'right': 5,
 'reprint': 0,
 'multi': 0,
 'study': 0,
 'longitudinal': 1,
 'examination': 0,
 'abortion': 6,
 'huang': 7,
 'paul': 3,
 'davies': 3,
 'chris': 7,
 'sibley': 4,
 'danny': 3,
 'osborne': 2,
 'abstract': 0,
 'although': 8,
 'ideology': 5,
 'that': 2,
 'highly': 0,
 'revere': 3,
 'women': 6,
 'conform': 2,
 'traditional': 0,
 'gender': 0,
 'cloak': 7,
 'superficially': 2,
 'positive': 2,
 'tone': 0,
 'place': 0,
 'upon': 2,
 'pedestal': 7,
 'inherently': 5,
 'restrictive': 6,
 'accordingly': 0,
 'because': 2,
 'paternalistic': 6,
 'beliefs': 2,
 'associate': 0,
 'with': 2,
 'base': 5,
 'idealization': 2,
 'roles': 0,
 'which': 0,
 'include': 0,
 'should': 2,
 'predict': 2,
 'people': 2,
 'data': 1,
 'from': 8,
 'nationwide': 5,
 'panel': 0,
 'show': 0,
 'hostile': 

In [90]:
s3 = boto3.client('s3')
serializedclusters = pickle.dumps(predicted_cluster)
s3.put_object(Bucket='inroads-test-bucket1',Key='myDictionary', Body=serializedclusters)

{'ResponseMetadata': {'RequestId': '67E2A2E7A67EDC17',
  'HostId': 'Ri+AT+SvNKg3pC06aHSiE6p26ULoqaFWQUU2qKUYcuXOvcP2g3ZQnZgNnegsvReaJcxVxbG4o08=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'Ri+AT+SvNKg3pC06aHSiE6p26ULoqaFWQUU2qKUYcuXOvcP2g3ZQnZgNnegsvReaJcxVxbG4o08=',
   'x-amz-request-id': '67E2A2E7A67EDC17',
   'date': 'Tue, 18 Aug 2020 23:44:18 GMT',
   'etag': '"541bbc9f1e752eaa261d9ac22a639f57"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"541bbc9f1e752eaa261d9ac22a639f57"'}

In [94]:
import boto3
s3 = boto3.client('s3')
object = s3.get_object(Bucket='inroads-test-bucket1',Key='myDictionary')
serializedObject = object['Body'].read()
myData = pickle.loads(serializedObject)

dict

In [95]:
centroid_map_df = pd.DataFrame(list(centroid_map.items()),columns = ['token','cluster']) 

## import saved cluster structures

In [70]:
centers = np.load("cluster_9_centers.npy")
clusters = np.load("cluster_9_clusters.npy")
centroid_map = pd.read_csv("cluster_9.csv", header=None)
centroid_map.columns = ["token", "cluster"]

In [76]:
from sklearn.neighbors import KDTree

top_words,tree = get_top_words(old_vocab, 20, centers, old_array)
print(top_words)

    Cluster #0     Cluster #1  Cluster #2   Cluster #3 Cluster #4  \
1    narrative    misoprostol        feel         wife       june   
2   literature   mifepristone        fear       father    october   
3     identity       surgical        harm      husband   december   
4    knowledge      pregnancy        what     daughter      april   
5    sexuality  complications        pain         aunt       july   
6     cultural        medical       shame    boyfriend     august   
7       ethics     dilatation       grief  grandmother   february   
8      concept        uterine       wrong      brother    morning   
9   discussion     ultrasound     disgust       sister    january   
10  complexity       referral        love       cousin      march   
11    feminism       cervical        evil         girl   saturday   
12    research  contraceptive        good       friend       week   
13    dialogue      antenatal       moral         baby      night   
14      gender       delivery     

In [458]:
old_array[1].shape

(300,)

In [459]:
test_vector = old_vectors[1][1]

In [460]:
test_vector.shape

(300,)

In [463]:
tree.query(old_vectors[1][1].reshape(1,-1), k=3)

(array([[0.        , 1.22595241, 1.23703717]]), array([[  1, 797, 222]]))

In [ ]:
from sklearn.cluster import KMeans
tree = KMeans(9).fit(old_vectors[1][1])
tree

In [267]:
## merge vectors with clusters

In [396]:
result = pd.merge(result, centroid_map, how='left', on=['token'])
training_data = pd.merge(old_vocab_df, centroid_map, how='left', on=['token'])
training_data.to_csv('/home/ec2-user/SageMaker/deploy/training_data.csv',index=False)

result['cluster'] = result.cluster.astype(str)
result.dtypes 
result.head()

token       object
vector      object
0          float64
1          float64
2          float64
            ...   
296        float64
297        float64
298        float64
299        float64
cluster     object
Length: 303, dtype: object

In [12]:
import boto3
bucket = 'sagemaker-word2vec-scikitlearn'
region = 'us-east-2'
s3_session = boto3.Session().resource('s3')
s3_session.Bucket(bucket).Object('train/train.csv').upload_file('cluster_9.csv')

In [13]:
import boto3
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

data_location = 'https://sagemaker-word2vec-scikitlearn.s3.us-east-2.amazonaws.com/train/train.csv'

train_data = pd.read_csv(data_location, header=None, names=None)

HTTPError: HTTP Error 403: Forbidden

## classification

In [397]:
result = result.dropna()
y = result.cluster
X = result.iloc[:, 2:302]

In [402]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
clf.predict(X_test)

clf.predict_proba(X_test)

clf.score(X_test, y_test)


0.6779661016949152

In [398]:
from sklearn.model_selection import train_test_split
#from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import TfidfTransformer
#from sklearn.naive_bayes import MultinomialNB
from sklearn import svm


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

lin_clf = svm.LinearSVC()
clf = lin_clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

In [399]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test,y_pred)*100
accuracy

66.94915254237289

given the two relative accuracies it appears that logistic regression is just as powerful as svm, thus we can continue with logistic regression for ease